In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

c:\users\playdata\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.models import load_model

In [3]:
from keras.applications import vgg16

In [4]:
pre_train = vgg16.VGG16()

In [17]:
pre_train.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [6]:
pre_train.save_weights("VGG_pre_train.h5")

In [7]:
pre_train.load_weights("VGG_pre_train.h5")

### MNIST Data Load

In [11]:
batch_size = 128
num_classes = 10
epochs = 5

In [12]:
img_rows, img_cols = 28, 28

In [13]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [14]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)


In [15]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


### pre-train VGG customize

In [16]:
for layer in pre_train.layers[1:-1]: ## layer 1 ~ 마지막 까지 weight가 변하지 않는다.
    layer.trainable = False

In [25]:
from keras.models import Input

input_image = Input(shape=(img_rows, img_cols, 3), name='image_input')

In [26]:
from keras.models import Model

output_pre_trian_model = pre_train(input_image)

x = Flatten(name='flatten')(output_vgg16_conv)
predictions = Dense(10, activation="softmax")(x)

pretrain_model_final = Model(input = input_image, output = predictions)

ValueError: Negative dimension size caused by subtracting 2 from 1 for 'vgg16_3/block5_pool/MaxPool' (op: 'MaxPool') with input shapes: [?,1,1,512].

In [25]:
pretrain_model_final.compile(loss='categorical_crossentropy', optimizer='Adam')

In [26]:
pretrain_model_final.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

ValueError: Error when checking input: expected input_1 to have shape (224, 224, 3) but got array with shape (28, 28, 1)